In [ ]:
!pip install snowflake-snowpark-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.7/419.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.7 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 4.2.0
    Uninstalling platformdirs-4.2.0:
      Successfully uninstalled platformdirs-4.2.0


In [ ]:
pip install statsmodels

In [ ]:
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from snowflake.snowpark.functions import year, month
from snowflake.snowpark.functions import col, avg, stddev_pop, corr
import numpy as np
import pandas as pd
from snowflake.snowpark.exceptions import SnowparkSQLException
import statsmodels.api as sm

In [ ]:
parameters = {
...    "account": "YEZEPEO-DATASCIENCE",
...    "user": "VNSHARED",
...    "password": "Columbia2024!",
...    "role": "SYSADMIN", # optional
...    "warehouse": "ANALYSIS", # optional
...   "database": "KRIS_DATA", # optional
...   "schema":"PUBLIC",  # optional
...  }

In [ ]:
session= Session.builder.configs(parameters).create()

# Household Debt

Full dataset - no p-values

In [ ]:
df_v = session.sql('select * from CONSOLIDATED_COPY')
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])
df_v = df_v.select("FECHAEMISION", "DIASCOMPENSACION")

In [ ]:
# Step 1: Extract year and month, then identify unique year-month combinations
df_v_with_year_month = df_v.with_column("YEAR", year(col("FECHAEMISION")))\
                          .with_column("MONTH", month(col("FECHAEMISION")))
unique_year_months = df_v_with_year_month.select("YEAR", "MONTH").distinct().collect()

# Step 2: Iterate and filter for each year-month combination
filtered_dfs = []
for ym_row in unique_year_months:
    year_value, month_value = ym_row["YEAR"], ym_row["MONTH"]
    df_filtered = df_v_with_year_month.filter((col("YEAR") == year_value) & (col("MONTH") == month_value))
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_debt = session.sql('select * from DEBT_COPY')
df_debt = df_debt.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_debt.columns])

In [ ]:
print(df_debt.columns)

['EXTRACTIONDATE', 'COUNTRY', 'YEAR', 'DATE', '"Debt(%GDP)"']


In [ ]:
# Extract year and month for 'FechaEmision' in df_v
df_v = df_v.withColumn('Year', year('FECHAEMISION'))
df_v = df_v.withColumn('Month', month('FECHAEMISION'))

# Extract year and month for 'DATE' in df_debt
df_debt = df_debt.withColumn('Year', year('DATE'))
df_debt = df_debt.withColumn('Month', month('DATE'))

# Join the dataframes on year and month
df_with_debt = df_v.join(df_debt, (df_v['Year'] == df_debt['YEAR']))

# Select the desired columns including 'Debt(%GDP)' from df_debt
# Add any other columns you need from df_v and df_debt
df_with_debt = df_with_debt.select(df_v['*'], df_debt['Debt(%GDP)'])

In [ ]:
# Upload as table to perform linear regression.
# Define the name of the table you want to create or overwrite in Snowflake
table_name = 'table_with_debt'

# Write the DataFrame to a new table in Snowflake
df_with_debt.write.mode('overwrite').save_as_table(table_name)

In [ ]:
tb = session.sql('select * from TABLE_WITH_DEBT ')
print(tb.columns)

['FECHAEMISION', 'DIASCOMPENSACION', '"l_s3zn_YEAR"', '"l_s3zn_MONTH"', '"Debt(%GDP)"']


In [ ]:
# Perform the linear regression
regression_query = """
SELECT
    REGR_SLOPE("DIASCOMPENSACION", "Debt(%GDP)") AS slope,
    REGR_INTERCEPT("DIASCOMPENSACION", "Debt(%GDP)") AS intercept,
    REGR_R2("DIASCOMPENSACION", "Debt(%GDP)") AS r_squared,
    REGR_COUNT("DIASCOMPENSACION", "Debt(%GDP)") AS sample_size,
    REGR_SXX("DIASCOMPENSACION", "Debt(%GDP)") AS sxx,
    REGR_SYY("DIASCOMPENSACION", "Debt(%GDP)") AS syy,
    REGR_SXY("DIASCOMPENSACION", "Debt(%GDP)") AS sxy
FROM
    table_with_debt;
"""

# Execute the query
try:
    regression_results = session.sql(regression_query).collect()
    # Print the results
    for result in regression_results:
        print(f"Slope: {result['SLOPE']}")
        print(f"Intercept: {result['INTERCEPT']}")
        print(f"R-squared: {result['R_SQUARED']}")
        print(f"Sample size: {result['SAMPLE_SIZE']}")
        print(f"SXX: {result['SXX']}")
        print(f"SYY: {result['SYY']}")
        print(f"SXY: {result['SXY']}")
except SnowparkSQLException as e:
    print(f"An error occurred: {e}")

Slope: -4.209309585823958
Intercept: 284.6847329766174
R-squared: 0.006584492425216656
Sample size: 395582638
SXX: 12889620695.605225
SYY: 34684830127735.266
SXY: -54256403951.64575


limited dataset - with p-value

limit 10000 observations in each year

In [ ]:
df_v=session.sql('select * from CONSOLIDATED_COPY')

In [ ]:
df_v = df_v.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_v.columns])

In [ ]:
# Step 1: Identify unique years
df_v_with_year = df_v.with_column("YEAR", year(col("FECHAEMISION")))
unique_years = df_v_with_year.select("YEAR").distinct().collect()

# Step 2: Iterate and filter for each year
filtered_dfs = []
for year_row in unique_years:
    year_value = year_row["YEAR"]
    df_filtered = df_v_with_year.filter(col("YEAR") == year_value).limit(10000)
    filtered_dfs.append(df_filtered)

# Step 3: Union results
final_df = filtered_dfs[0]
for df in filtered_dfs[1:]:
    final_df = final_df.union(df)

In [ ]:
df_debt = session.sql('select * from DEBT_COPY')

In [ ]:
df_debt = df_debt.select([F.col(col_name).alias(col_name.replace('"', '')) for col_name in df_debt.columns])

In [ ]:
df_debt = df_debt.to_pandas()
final_df=final_df.to_pandas()

In [ ]:
#Convert 'FechaEmision' and 'DATE' to datetime
final_df['FECHAEMISION'] = pd.to_datetime(final_df['FECHAEMISION'])
df_debt['DATE'] = pd.to_datetime(df_debt['DATE'])

# Extract the year from 'FechaEmision' and 'Year'
final_df['YEAR'] = final_df['FECHAEMISION'].dt.year
df_debt['YEAR'] = df_debt['YEAR'].astype(int)

# Merge the dataframes on the 'Year' column
df_with_debt = pd.merge(final_df, df_debt[['YEAR', 'Debt(%GDP)']], on='YEAR', how='inner')

# Drop the extra 'Year' column if you don't need it
df_with_debt.drop('YEAR', axis=1, inplace=True)

# Check the result
df_with_debt

,FECHAEXTRACCION,NUMERODOCUMENTO,NUMEROFOLIO,FECHAEMISION,FECHAVENCIMIENTO,FECHAREGISTRO,BUSINESSPARTNERID,CUENTACONTRATO,CLAVERECONCILIACION,CABECERA_FECHAHORACREACION,...,TRAMOMORA,INHABILITADO,CONCESIONARIA,TIPOCLIENTE,TIPOOPERACION,TIPODOCUMENTO,TIPOPRODUCTO,STATUSCOMPENSACION,STATUSCOMPENSACIONTMP,Debt(%GDP)
0,2024-01-15 13:52:32.627,6.360005e+10,1524147.0,2007-01-03,2007-01-22,2007-01-04,0010193118,001000206901,07004FA010AM,2021-04-21 18:16:46,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta Exenta,Tag,Pagada,Pagada,29.027133
1,2024-01-15 13:52:32.627,6.360006e+10,1613433.0,2007-01-17,2007-02-05,2007-01-17,0010393441,001100149165,07016FA040AI,2021-04-21 18:19:09,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,29.027133
2,2024-01-15 13:52:32.627,6.360006e+10,1613433.0,2007-01-17,2007-02-05,2007-01-17,0010393441,001100149165,07016FA040AI,2021-04-21 18:19:09,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,29.027133
3,2024-01-15 13:52:32.627,6.360006e+10,1596496.0,2007-01-11,2007-01-29,2007-01-12,0010767303,001100522963,07012SA032AB,2021-04-21 18:18:14,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta Exenta,Tag,Pagada,Pagada,29.027133
4,2024-01-15 13:52:32.627,6.360005e+10,1524551.0,2007-01-03,2007-01-22,2007-01-04,0010305954,001100062423,07004FA010AM,2021-04-21 18:16:46,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta Exenta,Tag,Pagada,Pagada,29.027133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169995,2024-01-15 13:52:32.627,1.160001e+10,157360.0,2006-03-13,2006-03-27,2006-03-14,0010882626,001100642185,06072FA030AA,2021-04-21 17:41:58,...,Pago,Cliente Tag,AVN,Natural,Peaje,Boleta,Tag,Castigado,Castigado,26.392439
169996,2024-01-15 13:52:32.627,1.640005e+10,745160.0,2006-06-12,2006-06-30,2006-06-12,0010703825,001100459501,06161FA033AH,2021-02-02 01:30:52,...,Pago,Cliente Tag,AVN,Natural,Intereses,Boleta,Tag,Pagada,Pagada,26.392439
169997,2024-01-15 13:52:32.627,1.640005e+10,73200.0,2006-06-07,2006-06-26,2006-06-07,0010744098,001100499760,06154FA020AA,2021-04-21 17:48:14,...,Pago,Cliente Tag,AVN,Juridico,Peaje,Factura,Tag,Castigado,Castigado,26.392439
169998,2024-01-15 13:52:32.627,1.400001e+10,154610.0,2006-03-13,2006-03-27,2006-03-13,0010830036,001100585678,06072FA030AR,2021-02-02 01:30:52,...,Pago,Cliente Tag,AVN,Natural,Gastos Cobranzas,Boleta,Tag,Pagada,Pagada,26.392439


In [ ]:
df_filtered = df_with_debt.dropna(subset=['Debt(%GDP)', 'DIASCOMPENSACION'])

# Calculate correlation
correlation = df_filtered['Debt(%GDP)'].corr(df_filtered['DIASCOMPENSACION'])

# Use statsmodels library for detailed statistics
X = sm.add_constant(df_filtered['Debt(%GDP)'])  # Adding a constant term to the predictor
y = df_filtered['DIASCOMPENSACION']

model = sm.OLS(y, X)
results = model.fit()

# Print summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:       DIASCOMPENSACION   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     478.9
Date:                Tue, 26 Mar 2024   Prob (F-statistic):          5.39e-106
Time:                        05:56:14   Log-Likelihood:            -1.1798e+06
No. Observations:              164681   AIC:                         2.360e+06
Df Residuals:                  164679   BIC:                         2.360e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        213.3601      4.567     46.721      0.0

In [ ]:
print("Model Coefficients:")
print(f"Intercept: {results.params[0]:.3f}")
print(f"Slope (DIASCOMPENSACION): {results.params[1]:.3f}")

print("\nModel P-values:")
print(f"P-value (Intercept): {results.pvalues[0]:.3f}")
print(f"P-value (DIASCOMPENSACION): {results.pvalues[1]:.3f}")

print("\nModel R-squared: {:.3f}".format(results.rsquared))
print("Model Standard Errors:")
print(f"Standard Error (Intercept): {results.bse[0]:.3f}")
print(f"Standard Error (DIASCOMPENSACION): {results.bse[1]:.3f}")

Model Coefficients:
Intercept: 213.360
Slope (DIASCOMPENSACION): -2.547

Model P-values:
P-value (Intercept): 0.000
P-value (DIASCOMPENSACION): 0.000

Model R-squared: 0.003
Model Standard Errors:
Standard Error (Intercept): 4.567
Standard Error (DIASCOMPENSACION): 0.116


In [ ]:
session.close

<bound method Session.close of <snowflake.snowpark.session.Session object at 0x7a863b9f3070>>